In [7]:
import numpy as np
import torch
from torch.nn import functional as F
import transformers
import pdb
from datasets import load_dataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from transformers import GPT2LMHeadModel
from transformers import GPT2TokenizerFast
from transformers import Trainer
from transformers import TrainingArguments

In [8]:
def get_dataset():
    train_dataset = load_dataset("rotten_tomatoes", split="train")
    eval_dataset = load_dataset("rotten_tomatoes", split="validation")
    return train_dataset, eval_dataset

In [15]:
def load_init_model_optimizer_tokenizer():
    pretrained_weights = 'gpt2'
    model = GPT2LMHeadModel.from_pretrained(pretrained_weights)
    tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
    # models are usually loaded in eval() mode, so set this to train()
    model.train()
    # initialize the optimizer
    optimizer = AdamW(model.parameters(), lr=1e-5)
    # using weight decay
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
    tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)

    return model, optimizer, tokenizer

In [10]:
"""
in model.generate you need to set return_dict_in_generate=False
to return torch.LongTensor
"""
def sampling_loop(model, input_ids, attention_mask, num_decode_steps=10):
    output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=num_decode_steps)

In [17]:
# get train and eval datasets
train_dataset, eval_dataset = get_dataset()
# get model, optimizer, tokenizer
model, optimizer, tokenizer = load_init_model_optimizer_tokenizer()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    tokenizer=tokenizer,                 # tokenizer
)

/Users/annhe/anaconda3/envs/agency/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/annhe/anaconda3/envs/agency/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`